In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from tqdm import tqdm
import MySQLdb
import re
import time

#웹드라이버 설정
driver = webdriver.Chrome() 
driver.implicitly_wait(3)

#사이트 이동
driver.get('https://www.jobplanet.co.kr/welcome/index')
actions = ActionChains(driver)

# 현재 창 핸들 저장
main_window = driver.current_window_handle

#search_doc = driver.find_element(By.ID, 'search_bar_search_query')

company_datas_list = []

#기업명 검색 

company_datas_list = []

for i in tqdm(range(len(company_names)), desc='loading'):
    driver.switch_to.window(main_window)
    
    search_doc = driver.find_element(By.ID, 'search_bar_search_query')
    search_doc.clear()
    search_doc.send_keys(company_names[i])
    search_doc.send_keys(Keys.ENTER)

    company_datas = []

    try:

        # 기업 정보 창으로 이동
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "text-small1.text-gray-400"))
        )
        element.click()

        time.sleep(1)

        all_windows = driver.window_handles

        # 새 창으로 전환 (기존 핸들과 다른 창으로 전환)
        for window_handle in all_windows:
            if window_handle != main_window:
                driver.switch_to.window(window_handle)
                break

                
        try:
            time.sleep(2)
            review = driver.find_element(By.ID, "premiumReviewStatistics").find_element(By.CLASS_NAME, "flex")
            review_list = review.text.split("\n")
            review_data = []
                
            # 전체 평점
            rating = ['전체 평점', review_list[0]]
            review_data.append(rating)
            # 복지 및 급여
            service = review_list[1:3]
            review_data.append(service)
            # 업무와 삶의 균형
            balance = review_list[3:5]
            review_data.append(balance)
            # 사내 문화
            culture = review_list[5:7]
            review_data.append(culture)
            # 기업 추천율
            recommend = review_list[-6:-4][::-1]
            review_data.append(recommend)
            # 성장 가능성
            potential = review_list[-2:][::-1]
            review_data.append(potential)

            #뉴스룸창
            news_tab = driver.find_element(By.CLASS_NAME, "txt.jf_b1.jf_gray700.relative.inline-block.text-center") 
            news_tab.click()

            news_data = []

            #기업 정보
            com_inform = driver.find_element(By.CLASS_NAME, "basic_info_sec")
            detail = com_inform.text.split("\n")
            #설립연도
            date = ['설립연도' , detail[6]]
            news_data.append(date)
            #기업 형태
            size = ['기업 형태' , detail[2]]
            news_data.append(size)
            #사원수
            pop = ['사원수' , detail[4]]
            news_data.append(pop)

            #매출
            if detail[11] == '':
                keyword = driver.find_element(By.CLASS_NAME, "info_btn_group").find_element(By.CLASS_NAME, "btn_info_more.ic_arrow_light_down")
                drag = com_inform
                drop = keyword
                actions.drag_and_drop(drag, drop).perform()

                keyword.click()

                com_inform = driver.find_element(By.CLASS_NAME, "basic_info_sec")
                detail = com_inform.text.split("\n")
                sales = ['매출', detail[11]]
                news_data.append(sales)
            else:
                sales = ['매출', detail[11]]
                news_data.append(sales)


            company_data = [f"{company_names[i]}", news_data, review_data]
            company_datas.append(company_data)

            driver.close()
            
            # 메인 창으로 돌아가기
            driver.switch_to.window(main_window)

        except:
            review_tab = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "viewReviews")))
            review_tab.click()
            
            time.sleep(2)
            review = driver.find_element(By.ID, "premiumReviewStatistics").find_element(By.CLASS_NAME, "flex")
            review_list = review.text.split("\n")
            review_data = []
                
            # 전체 평점
            rating = ['전체 평점', review_list[0]]
            review_data.append(rating)
            # 복지 및 급여
            service = review_list[1:3]
            review_data.append(service)
            # 업무와 삶의 균형
            balance = review_list[3:5]
            review_data.append(balance)
            # 사내 문화
            culture = review_list[5:7]
            review_data.append(culture)
            # 기업 추천율
            recommend = review_list[-6:-4][::-1]
            review_data.append(recommend)
            # 성장 가능성
            potential = review_list[-2:][::-1]
            review_data.append(potential)

            #뉴스룸창
            news_tab = driver.find_element(By.CLASS_NAME, "txt.jf_b1.jf_gray700.relative.inline-block.text-center") 
            news_tab.click()
            time.sleep(2)
            news_data = []

            #기업 정보
            com_inform = driver.find_element(By.CLASS_NAME, "basic_info_sec")
            detail = com_inform.text.split("\n")
            #설립연도
            date = ['설립연도' , detail[6]]
            news_data.append(date)
            #기업 형태
            size = ['기업 형태' , detail[2]]
            news_data.append(size)
            #사원수
            pop = ['사원수' , detail[4]]
            news_data.append(pop)
            #매출

            if detail[11] == '':
                keyword = driver.find_element(By.CLASS_NAME, "info_btn_group").find_element(By.CLASS_NAME, "btn_info_more.ic_arrow_light_down")
                drag = com_inform
                drop = keyword
                actions.drag_and_drop(drag, drop).perform()

                keyword.click()

                com_inform = driver.find_element(By.CLASS_NAME, "basic_info_sec")
                detail = com_inform.text.split("\n")
                sales = ['매출', detail[11]]
                news_data.append(sales)
            else:
                sales = ['매출', detail[11]]
                news_data.append(sales)

            company_data = [f"{company_names[i]}", news_data, review_data]
            company_datas.append(company_data)

            driver.close()
            
            # 메인 창으로 돌아가기
            driver.switch_to.window(main_window)
    
    except:
        company_datas = [[f"{company_names[i]}", [], []]]
        print(f'*{company_names[i]}* (검색 결과 없음)')

        driver.switch_to.window(main_window)

    company_datas_list.append(company_datas)

driver.quit()

company_datas_list

loading: 100%|██████████| 1/1 [00:22<00:00, 22.07s/it]


[[['뤼튼테크놀로지스',
   [['설립연도', '2021.04.13'], ['기업 형태', '중소기업'], ['사원수', '100명'], ['매출', '-']],
   [['전체 평점', '2.1'],
    ['복지 및 급여', '3.8'],
    ['업무와 삶의 균형', '2.1'],
    ['사내문화', '3.0'],
    ['기업 추천율', '27%'],
    ['성장 가능성', '20%']]]]]

## 데이터 베이스 저장된 데이터 가져오기

## 데이터 베이스 연결

In [14]:
import pymysql
# MySQL 데이터베이스 연결 함수
def connect_to_db():
    return pymysql.connect(
    host="roseda.cj6s66ayq6e8.ap-northeast-2.rds.amazonaws.com",
    user="fo",
    password='1234',
    database="folium"
)
# 데이터베이스 연결


try:
    conn = connect_to_db()
    cursor = conn.cursor()
    # 1. 회사 정보 테이블 생성 (established_date를 NULL 허용하도록 수정)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS company_info (
            id INT AUTO_INCREMENT PRIMARY KEY,
            company_name VARCHAR(100) NOT NULL UNIQUE,  -- UNIQUE 제약 조건
            established_date DATE NULL,  -- NULL 허용
            company_type VARCHAR(50),
            employees INT,
            revenue VARCHAR(50)
        )
    ''')
    # 2. 회사 평점 테이블 생성 (회사 이름에 UNIQUE 제약 조건 추가)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS company_rating (
            id INT AUTO_INCREMENT PRIMARY KEY,
            company_name VARCHAR(100) NOT NULL,
            overall_rating FLOAT,
            welfare_salary FLOAT,
            work_life_balance FLOAT,
            company_culture FLOAT,
            recommendation FLOAT,
            growth_potential FLOAT
        )
    ''')

    # 3. 주어진 데이터 (여러 회사 정보)
    for company_data in company_datas_list:
        # 4. 여러 회사 데이터를 삽입하는 반복문
        for company in company_data:
            company_name = company[0]  # 회사 이름
            info = company[1]  # 회사 정보 리스트
            ratings = company[2]  # 평점 정보 리스트
            # 5. 회사 정보 추출
            if info:  # 회사 정보가 있는 경우
                established_date = info[0][1] if len(info[0]) > 1 else None  # '2008.02.04' 또는 None
                company_type = info[1][1] if len(info[1]) > 1 else None  # '중소기업'

                # 사원수 정보 처리
                employees_str = info[2][1] if len(info[2]) > 1 else None  # '16명' 또는 None
                if employees_str and employees_str != '-':
                    employees = int(employees_str.replace('명', '').strip())  # '16명' -> 16
                else:
                    employees = None  # 잘못된 경우 None으로 처리


                #employees = int(info[2][1].replace('명', '')) if len(info[2]) > 1 else None  # '16명' -> 16

                revenue = info[3][1] if len(info[3]) > 1 else None  # '13억 원 (2012)'
            else:  # 회사 정보가 없을 경우 기본값 설정
                established_date = None
                company_type = None
                employees = None
                revenue = None
            # 6. 회사 정보 삽입 (중복된 경우 덮어쓰기)
            cursor.execute('''
                INSERT INTO company_info (company_name, established_date, company_type, employees, revenue)
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    established_date = VALUES(established_date),
                    company_type = VALUES(company_type),
                    employees = VALUES(employees),
                    revenue = VALUES(revenue)
            ''', (company_name, established_date, company_type, employees, revenue))
            # 7. 평점 정보 추출 및 삽입
            if ratings:  # 평점 정보가 있는 경우
                overall_rating = float(ratings[0][1])  # '2.4'
                welfare_salary = float(ratings[1][1])  # '2.2'
                work_life_balance = float(ratings[2][1])  # '3.1'
                company_culture = float(ratings[3][1])  # '2.6'
                recommendation = float(ratings[4][1].replace('%', '')) / 100  # '35%' -> 0.35
                growth_potential = float(ratings[5][1].replace('%', '')) / 100  # '26%' -> 0.26
                # 회사 평점 정보 삽입 (중복된 경우 덮어쓰기)
                cursor.execute('''
                    INSERT INTO company_rating (company_name, overall_rating, welfare_salary, work_life_balance, company_culture, recommendation, growth_potential)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE
                        overall_rating = VALUES(overall_rating),
                        welfare_salary = VALUES(welfare_salary),
                        work_life_balance = VALUES(work_life_balance),
                        company_culture = VALUES(company_culture),
                        recommendation = VALUES(recommendation),
                        growth_potential = VALUES(growth_potential)
                ''', (company_name, overall_rating, welfare_salary, work_life_balance, company_culture, recommendation, growth_potential))
        # 8. 변경 사항 저장
        conn.commit()
except pymysql.MySQLError as e:
    print(f"Error executing SQL: {e}")
    conn.rollback()
finally:
    # 9. 데이터베이스 연결 종료
    if conn:
        conn.close()

In [85]:
import pymysql
def connect_to_db():
    return pymysql.connect(
    host="roseda.cj6s66ayq6e8.ap-northeast-2.rds.amazonaws.com",
    user="fo",
    password='1234',
    database="folium"
)

try:
    conn = connect_to_db()
    cursor = conn.cursor()
    # 1. 새로운 테이블 생성
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS company_combined (
            company_name VARCHAR(100),
            established_date DATE,
            company_type VARCHAR(50),
            employees INT,
            revenue VARCHAR(50),
            overall_rating FLOAT,
            welfare_salary FLOAT,
            work_life_balance FLOAT,
            company_culture FLOAT,
            recommendation FLOAT,
            growth_potential FLOAT
        )
    ''')
    # 2. 조인 쿼리 실행 및 결과를 새로운 테이블에 삽입
    query = '''
    INSERT INTO company_combined (
        company_name,
        established_date,
        company_type,
        employees,
        revenue,
        overall_rating,
        welfare_salary,
        work_life_balance,
        company_culture,
        recommendation,
        growth_potential
    )
    SELECT
        ci.company_name,
        ci.established_date,
        ci.company_type,
        ci.employees,
        ci.revenue,
        cr.overall_rating,
        cr.welfare_salary,
        cr.work_life_balance,
        cr.company_culture,
        cr.recommendation,
        cr.growth_potential
    FROM
        (select distinct * from company_info) ci, (select distinct * from company_rating) cr
    where
        ci.company_name = cr.company_name ;
    '''
    cursor.execute(query)
    conn.commit()  # 변경 사항 저장
    print("조인 결과가 company_combined 테이블에 성공적으로 삽입되었습니다.")
except pymysql.MySQLError as e:
    print(f"Error executing SQL: {e}")
    conn.rollback()
finally:
    if conn:
        conn.close()

조인 결과가 company_combined 테이블에 성공적으로 삽입되었습니다.


## 중복 데이터 제거

In [18]:
import pymysql
def connect_to_db():
    return pymysql.connect(
    host="roseda.cj6s66ayq6e8.ap-northeast-2.rds.amazonaws.com",
    user="fo",
    password='1234',
    database="folium"
)

try:
    conn = connect_to_db()
    cursor = conn.cursor()
    # 1. 중복된 company_name을 제거하기 위한 쿼리
    delete_query = '''
    DELETE c1 FROM company_combined c1
    JOIN company_combined c2
    ON c1.company_name = c2.company_name
    WHERE c1.company_info_id > c2.company_info_id;
    '''
    cursor.execute(delete_query)
    conn.commit()  # 변경 사항 저장
    print("중복된 company_name이 제거되었습니다.")
except pymysql.MySQLError as e:
    print(f"Error executing SQL: {e}")
    conn.rollback()
finally:
    if conn:
        conn.close()

중복된 id가 제거되었습니다.


## 데이터 베이스 저장된 데이터 가져오기

In [29]:
import pandas as pd
import pymysql

def connect_to_db():
    return pymysql.connect(
    host="roseda.cj6s66ayq6e8.ap-northeast-2.rds.amazonaws.com",
    user="fo",
    password='1234',
    database="folium"
)

def load_company_combined_to_dataframe():
    try:
        conn = connect_to_db()
        # SQL 쿼리 작성
        query = "SELECT * FROM company_combined;"
        # 데이터프레임으로 로드
        df = pd.read_sql(query, conn)
        return df
    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
        return None
    finally:
        if conn:
            conn.close()
# 데이터프레임 가져오기
company_combined_df = load_company_combined_to_dataframe()
# 결과 출력
company_combined_df


/tmp/ipykernel_49612/1934477392.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,company_name,established_date,company_type,employees,revenue,overall_rating,welfare_salary,work_life_balance,company_culture,recommendation,growth_potential
0,몰로코,2013-11-01,외국계 (외국 투자기업),640.0,"5,000억 원 (2021)",4.3,4.1,3.7,3.9,0.92,0.92
1,플래티어,2005-07-01,중소기업,138.0,342억 8천만 원 (2019),3.1,3.3,3.3,3.1,0.54,0.29
2,한국축산데이터,2017-11-20,중소기업,100.0,-,3.0,2.8,3.9,2.9,0.40,0.35
3,뤼이드,2014-05-01,중소기업,150.0,77억 원 (2023),3.1,3.4,3.6,2.8,0.36,0.36
4,매스프레소,2015-06-22,중소기업,280.0,-,3.4,3.3,2.9,3.6,0.50,0.34
...,...,...,...,...,...,...,...,...,...,...,...
173,딥네츄럴,2017-04-21,중소기업,14.0,-,3.7,2.3,3.8,4.3,0.39,0.17
174,넥스트칩,2019-01-02,중소기업,175.0,103억 8천만 원 (2020),2.6,2.9,3.0,2.2,0.26,0.14
175,라스테크,0000-00-00,-,NaN,-,1.6,1.5,1.7,1.4,0.00,0.00
176,원더풀플랫폼,2016-01-21,-,NaN,-,2.1,2.4,2.8,2.3,0.15,0.13
